In [1]:
import fs
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from varclushi import VarClusHi
from optbinning import BinningProcess

(CVXPY) Sep 24 11:55:57 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Sep 24 11:55:57 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:
PROCESSED_PROJECT_DIR = fs.open_fs("../../data/processed/project")
TRAIN_FILE_NAME = PROCESSED_PROJECT_DIR.getsyspath("train.csv")

In [3]:
train_df = pd.read_csv(TRAIN_FILE_NAME)

### Checking Variance

In [4]:
categoric_features = ['BAD', 'REASON', 'JOB']
train_df[categoric_features] = train_df[categoric_features].apply(lambda col: col.astype('category'))

In [5]:
numeric_features = train_df.select_dtypes(include=['int', 'float']).columns.tolist()

In [6]:
scaler = MinMaxScaler(feature_range=(10, 100))
train_df[numeric_features] = scaler.fit_transform(train_df[numeric_features])

In [7]:
vt = VarianceThreshold(threshold=0.15)

In [8]:
vt.fit(train_df[numeric_features])

VarianceThreshold(threshold=0.15)

In [9]:
varianza_peq = [v for v,nu in zip(train_df[numeric_features].columns,vt.get_support()) if not nu]
len(varianza_peq)

0

### Applying BinningProcess

In [10]:
X = train_df.drop(columns=['BAD'])
y = train_df['BAD']

In [11]:
categoric_features = ['REASON', 'JOB']

In [12]:
label_encoders = {}

for column in categoric_features:
    label_encoders[column] = LabelEncoder()
    X[column] = label_encoders[column].fit_transform(X[column])

In [13]:
vc = VarClusHi(df = X)
vc.varclus()

In [14]:
rs = vc.rsquare
rs = rs.sort_values(by=['Cluster', 'RS_Ratio']).reset_index(drop = True)
rs['id'] = rs.groupby('Cluster').cumcount()+1

In [15]:
rs

,Cluster,Variable,RS_Own,RS_NC,RS_Ratio,id
0,0,VALUE,0.884121,0.106870,0.129745,1
1,0,MORTDUE,0.804553,0.096080,0.216221,2
2,0,LOAN,0.290790,0.015217,0.720169,3
3,0,JOB,0.086597,0.007061,0.919898,4
4,1,NINQ,0.499697,0.015778,0.508323,1
5,1,DEBTINC,0.284991,0.016091,0.726703,2
6,1,REASON,0.226929,0.004625,0.776663,3
7,1,YOJ,0.231415,0.017636,0.782384,4
8,2,DELINQ,0.599143,0.016819,0.407715,1
9,2,DEROG,0.599143,0.020065,0.409065,2


In [16]:
variable_names = list(X.columns)

binning_process = BinningProcess(variable_names)
binning_process.fit(X, y)
binning_process.information(print_level=1)

optbinning (Version 0.19.0)
Copyright (c) 2019-2024 Guillermo Navas-Palencia, Apache License 2.0

  Statistics
    Number of records                   2973
    Number of variables                   12
    Target type                       binary

    Number of numerical                   12
    Number of categorical                  0
    Number of selected                    12

  Time                                0.4491 sec



In [17]:
df_bins_num  = binning_process.summary().sort_values(by = 'iv', ascending = False)
df_bins_num  = df_bins_num[df_bins_num['dtype'] == 'numerical']
df_bins_num.rename(columns = {'name': 'Variable'}, inplace = True)
df_bins_num = df_bins_num.merge(rs, how = 'left', on = 'Variable')
df_bins_num = df_bins_num.sort_values(by=['Cluster', 'iv'], ascending = [True, False]).reset_index(drop = True)
df_bins_num['id'] = rs.groupby('Cluster').cumcount()+1

In [18]:
df_bins_num

,Variable,dtype,status,selected,n_bins,iv,js,gini,quality_score,Cluster,RS_Own,RS_NC,RS_Ratio,id
0,LOAN,numerical,OPTIMAL,True,7,0.219406,0.025989,0.217537,0.331389,0,0.290790,0.015217,0.720169,1
1,VALUE,numerical,OPTIMAL,True,7,0.16649,0.019847,0.182063,0.05667,0,0.884121,0.106870,0.129745,2
2,JOB,numerical,OPTIMAL,True,4,0.086211,0.010606,0.13631,0.218258,0,0.086597,0.007061,0.919898,3
3,MORTDUE,numerical,OPTIMAL,True,6,0.07867,0.009762,0.153565,0.0296,0,0.804553,0.096080,0.216221,4
4,DEBTINC,numerical,OPTIMAL,True,6,0.578483,0.066971,0.327112,0.073815,1,0.284991,0.016091,0.726703,1
5,NINQ,numerical,OPTIMAL,True,5,0.157684,0.019246,0.199603,0.143144,1,0.499697,0.015778,0.508323,2
6,YOJ,numerical,OPTIMAL,True,5,0.058729,0.007286,0.127383,0.026038,1,0.231415,0.017636,0.782384,3
7,REASON,numerical,OPTIMAL,True,2,0.008347,0.001043,0.042372,0.018633,1,0.226929,0.004625,0.776663,4
8,DELINQ,numerical,OPTIMAL,True,3,0.558981,0.064127,0.319013,0.368205,2,0.599143,0.016819,0.407715,1
9,DEROG,numerical,OPTIMAL,True,2,0.374568,0.043935,0.224687,0.278902,2,0.599143,0.020065,0.409065,2


IV > 0.15

- 0: LOAN, VALUE
- 1: DEBTINC, NINQ
- 2: DELINQ, DEROG
- 3: CLAGE